In [19]:
import requests
import urllib.request
import pandas as pd
import sqlite3
import csv
from bs4 import BeautifulSoup

## get a list of PA state park/forest from wiki
121 state parks, and 20 state forests

In [3]:
# get the url
url_park = 'https://en.wikipedia.org/wiki/List_of_Pennsylvania_state_parks'
response_park = requests.get(url_park)
print(response.status_code) 

soup = BeautifulSoup(response_park.text,"html.parser")

200


In [4]:
# find the table in need: first sortable table 
table = soup.findAll('table',{"class":"wikitable sortable"})[0]
# get table rows
tr = table.findAll(['tr']) # a total of 121 parks [& header]


# write info from the table to a csv file
csvFile = open("state_park_PA.csv",'wt',newline='', encoding='utf-8')
writer = csv.writer(csvFile)  
try:   
    for cell in tr:
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]
        writer.writerow(th_data+row)      
        
finally:   
    csvFile.close()

### repeat for state forest

In [5]:
url_forest = 'https://en.wikipedia.org/wiki/List_of_Pennsylvania_state_forests'
response_forest = requests.get(url_forest)
print(response_forest.status_code)

soup = BeautifulSoup(response_forest.text,"html.parser")

table = soup.findAll('table',{"class":"wikitable sortable"})[0]
tr = table.findAll(['tr']) # a total of 20 forests [& header]
csvFile = open("state_forest_PA.csv",'wt',newline='', encoding='utf-8')
writer = csv.writer(csvFile)  
try:   
    for cell in tr:
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]
        writer.writerow(th_data+row)      
        
finally:   
    csvFile.close()

200


In [17]:
df_forest = pd.read_csv('state_forest_PA.csv')
df_forest

,State Forest Name,County,Areaacres (ha),Founded,Remarks
0,Bald Eagle,"Centre, Clinton, Mifflin, Union, & Snyder","193,424 acres (78,280 ha)",NaN,NaN
1,Buchanan,"Bedford, Franklin, & Fulton","75,000 acres (30,350 ha)",NaN,NaN
2,Clear Creek,"Forest, Jefferson, & Venango","13,266 acres (5,369 ha)","1919, as Kittanning State Forest",Name changed 2007
3,Cornplanter,"Crawford, Forest, & Warren","1,256 acres (508 ha)",NaN,NaN
4,Delaware,Pike & Monroe,"83,519 acres (33,799 ha)",NaN,NaN
5,Elk,"Cameron, Clinton, Elk, McKean, & Potter","200,000 acres (80,940 ha)",1900,NaN
6,Forbes,"Fayette, Somerset, & Westmoreland","over 50,000 acres (20,230 ha)",NaN,NaN
7,Gallitzin,"Bedford, Cambria, Indiana, & Somerset","15,336 acres (6,206 ha)",NaN,NaN
8,Loyalsock,"Bradford, Lycoming, & Sullivan","114,552 acres (46,360 ha)","July 1, 2005",Replaced Wyoming State Forest
9,Michaux,"Adams, Cumberland, & Franklin","over 85,000 acres (34,400 ha)",NaN,NaN


In [23]:
con = sqlite3.connect("state_forest_PA_google_place_id.db")
df = pd.read_sql_query("SELECT id, name, address FROM state_forest", con)
con.close()
df

,id,name,address
0,ChIJVdp4ZCi8zokR-BN2PkJVZAQ,Bald Eagle State Forest,"Bald Eagle State Forest, Walker Township, PA 1..."
1,ChIJp6qqqvpbyokRlThh6d1zxHo,Buchanan State Forest,"Clearville, PA 15535, United States"
2,ChIJG9pKq3nezIkRf8e_mHrUZ0s,Clear Creek State Forest,"Sigel, PA 15860, United States"
3,ChIJvyueE8pEzYkRoVBwUV2-kuw,Cornplanter State Forest,"North Warren, PA 16365, United States"
4,ChIJU1VVVZWfxIkRhUhK_7Y6xJo,Delaware State Forest,"Dingmans Ferry, PA 18328, United States"
5,ChIJcRuHo6fKzYkRj01hdSwOfu4,Elk State Forest,"Elk State Forest, Benezette Township, PA 15868..."
6,ChIJM4TClPcny4kRhVB49yVSEI0,Forbes State Forest,"Forbes State Forest, Cook Township, PA 15677, USA"
7,ChIJa4nSqlYGy4kREMPuwlu9T78,Gallitzin State Forest,"Gallitzin State Forest, Ogle Township, PA 1596..."
8,ChIJOxLy42Tsz4kRxcGLnp5ff0w,Loyalsock State Forest,"Loyalsock State Forest, Fox Township, PA 18616..."
9,ChIJk472rC-hyYkRUHOh-AkNBvQ,Michaux State Forest,"10099 Lincoln Way E, Fayetteville, PA 17222, U..."


In [26]:
con = sqlite3.connect("state_park_PA_google_place_id.db")
df_park = pd.read_sql_query("SELECT id, name, address FROM state_park", con)
con.close()
df_park

,id,name,address
0,ChIJFbEYjJqUNIgRPLxC2ZWBxe4,Allegheny Islands State Park,"Cheswick, PA 15024, United States"
1,ChIJqV69o1LQxIkRcH7deU6CJFg,Archbald Pothole State Park,"960 Scranton Carbondale Hwy, Archbald, PA 1840..."
2,ChIJPUAh3FWRzokRTusj8NUp8dk,Bald Eagle State Park,"149 Main Park Rd, Howard, PA 16841, United States"
3,ChIJF9CkZ8FRxIkRH3KWMSBmvBY,Beltzville State Park,"2950 Pohopoco Dr, Lehighton, PA 18235, United ..."
4,ChIJ0Xs1BMSnzYkR6GLE86aPuT8,Bendigo State Park,"533 Johnsonburg Rd, St Marys, PA 15857, United..."
...,...,...,...
116,ChIJdfywo6xXwYkRuS111bIPKqU,Washington Crossing Historic Park,"1112 River Rd, Washington Crossing, PA 18977, ..."
117,ChIJk00-rKOrzokRS_4BHwJvaUM,Whipple Dam State Park,"3746 Laurel Run Rd, Petersburg, PA 16669, Unit..."
118,ChIJod2hF0Srx4kRNsFIgSQ44JQ,White Clay Creek Preserve,"404 Sharpless Rd, Landenberg, PA 19350, United..."
119,ChIJaeKPQD_zz4kRy4c8TvnwGIg,Worlds End State Park,"82 Cabin Bridge Rd, Forksville, PA 18616, Unit..."
